Fieldtrip: Audio Vs Visual Vs Motor
===================================

Lead authors: Hadi Zaatiti <hadi.zaatiti@nyu.edu>

- Description of experiment

The `Audio vs Visual vs Motor` code experiment in `Psychtoolbox` can be found here:

[Auditory vs Visual vs Motor](https://github.com/Hzaatiti/meg-pipeline/blob/main/experiments/psychtoolbox/auditory-vs-visual/auditory_vs_visual.m)

Importing data 
--------------

The data used in this notebook is hosted on `NYU BOX`. Permissions are given upon request.

- Install the BOX app from [here](https://www.box.com/resources/downloads)
- Set an environment variable with name `MEG_DATA` to the path of the Data folder e.g.,
    - `C:\Users\user_name\Box\MEG\Data`
    - or `C:\Users\user_name\Box\Data


MATLAB setup
------------

Make sure that:
- Fieldtrip is installed in MATLAB
- Add to MATLAB path the custom-made functions for NYUAD MEG lab found [here](https://github.com/Hzaatiti/meg-pipeline/tree/main/pipeline/field_trip_pipelines/matlab_functions)

Each experiment run using the KIT system generates a `.con` file and two or more .mrk files.

Data acquisition
----------------

The used data is acquired from a single subject with 150 trials of each condition type (visual, motor, auditory).



In [ ]:
%% FieldTrip pipeline for audio-visual-motor experiment
% Author: Hadi Zaatiti <hadi.zaatiti@nyu.edu>


clear;

% Reminder of stimulus types:

% 1 is visual stimulus = ch224
% 2 is auditory stimulus = ch225
% 3 is motor button = ch226

% Read the environment variable to NYU BOX
MEG_DATA_FOLDER = getenv('MEG_DATA');


% Define paths
TASK_NAME = 'audio-visual-motor';
SYSTEM = 'meg-kit';
SUB_ID = 'sub-001';
LASER_DEVICE = 'laser-scan';


% Construct the directory path
DATA_FOLDER_PATH = fullfile(MEG_DATA_FOLDER, TASK_NAME, SUB_ID, SYSTEM);

% List all .con files with the prefix 'sub-001'
filePattern = fullfile(DATA_FOLDER_PATH, [SUB_ID,'*.con']);
conFiles = dir(filePattern);


% Display the file names
disp('Found .con files:');
for k = 1:length(conFiles)
    disp(conFiles(k).name);
end


filePattern_mrk = fullfile(DATA_FOLDER_PATH, '*.mrk');

mrkFiles = dir(filePattern_mrk);

% Construct the directory path
DATA_FOLDER_PATH_LASER = fullfile(MEG_DATA_FOLDER, TASK_NAME, SUB_ID, LASER_DEVICE);

filePattern_laser_surface = fullfile(DATA_FOLDER_PATH_LASER,  [SUB_ID,'*basic-surface.txt']);
filePattern_laser_stylus = fullfile(DATA_FOLDER_PATH_LASER,  [SUB_ID,'*stylus-points.txt']);

laser_points = dir(filePattern_laser_surface);
laser_surf = dir(filePattern_laser_stylus);


APPLY_FILTERS = false;


We will skip filtering and see the analysis results with and without.

In [ ]:
%% Filtering data

if APPLY_FILTERS
    % Notch filter the data at 50 Hz
    cfg = [];
    cfg.bsfilter = 'yes';
    cfg.bsfreq = [49 51]; % Notch filter range
    combinedData = ft_preprocessing(cfg, combinedData);

    % Band-pass filter the data
    cfg = [];
    cfg.bpfilter = 'yes';
    cfg.bpfreq = [4 40]; % Band-pass filter range
    cfg.bpfiltord = 4;   % Filter order
    combinedData = ft_preprocessing(cfg, combinedData);
    
    disp('Filtering operations complete on combined data.');
end

In [ ]:
%%

% Initialize FieldTrip configuration
cfg = [];
cfg.coilaccuracy = 0;

% Cell array to store preprocessed data
dataList = {};

% Loop through all .con files
for k = 1:length(conFiles)
    % Construct the full path for the current .con file
    conFile = fullfile(DATA_FOLDER_PATH, conFiles(k).name);
    
    % Set the dataset in the configuration
    cfg.dataset = conFile;
    
    % Preprocess the MEG data
    fprintf('Processing file: %s\n', conFiles(k).name);
    dataList{k} = ft_preprocessing(cfg); % Store preprocessed data in the list
end

% Concatenate all preprocessed data
fprintf('Concatenating all preprocessed data...\n');
combinedData = ft_appenddata([], dataList{:});

% Display a message when concatenation is complete
disp('Data concatenation complete.');

In [ ]:
%% Define trials and segmentation of the data


previewTrigger = combinedData.trial{1}(225, :);

threshold = (max(previewTrigger) + min(previewTrigger)) / 2;
    
trigger_channels = [225, 226, 227];

TRIALS_DEF = cell(length(conFiles), length(trigger_channels));
TRIALS = cell(length(conFiles), length(trigger_channels));

for fileIdx = 1:length(conFiles)

    for chIdx = 1:length(trigger_channels)
        cfg = [];
        conFile = fullfile(DATA_FOLDER_PATH, conFiles(fileIdx).name);
        cfg.dataset  = conFile;
        cfg.trialdef.eventvalue = 1; % placeholder for the conditions
        cfg.trialdef.prestim    = 0.5; % 1s before stimulus onset
        cfg.trialdef.poststim   = 1.2; % 1s after stimulus onset
        cfg.trialfun = 'ft_trialfun_general';
        cfg.trialdef.chanindx = trigger_channels(chIdx);
        cfg.trialdef.threshold = threshold;
        cfg.trialdef.eventtype = 'combined_binary_trigger'; % this will be the type of the event if combinebinary = true
        cfg.trialdef.combinebinary = 1;
        cfg.preproc.baselinewindow = [-0.2 0];
        cfg.preproc.demean     = 'yes';
    
        % Define trials for the current channel and dataset
        TRIALS_DEF{fileIdx, chIdx} = ft_definetrial(cfg);

        % Preprocess trials for the current channel and dataset
        TRIALS{fileIdx, chIdx} = ft_preprocessing(TRIALS_DEF{fileIdx, chIdx});
    end
end

In [ ]:
%% Trials Concantenation

TRIALS_STIM = cell( length(trigger_channels),1);

for chIdx = 1:length(trigger_channels)
            cfg = [];
            TRIALS_STIM{chIdx} = ft_appenddata(cfg, TRIALS{1,chIdx}, TRIALS{2,chIdx}, TRIALS{3, chIdx});
end




In [ ]:
%% Visual Inspection ALTL

TRIALS_STIM_REJ = cell( length(trigger_channels),1);

for tr = 1:length(TRIALS_STIM)
    
    cfg = [];
    cfg.method='summary';
    cfg.channel = {'AG*'};
    TRIALS_STIM_REJ{tr} = ft_rejectvisual(cfg, TRIALS_STIM{tr});

end

save TRIALS_STIM_REJ TRIALS_STIM_REJ

In [ ]:
%% Averaging

AVG_TRIALS = cell( length(trigger_channels),1);

for tr = 1:length(AVG_TRIALS)

    cfg = [];

    AVG_TRIALS{tr} = ft_timelockanalysis(cfg, TRIALS_STIM_REJ{tr});

end

In [ ]:
%% Get KIT Sensors

kit_layout = create_kit_layout(conFile);

figure('Position', [100, 100, 1000, 800]); % Adjust the width and height (1000 and 800) as needed
ft_plot_layout(kit_layout, 'box', 1);



In [ ]:
%% Plotting in space

% for a single trial type, for each channel, average over time the trial
% and plot the average value on the helmet

% You can still see the time behavior when clicking on one sensor

for tr = 1:length(AVG_TRIALS)

    cfg = [];
    cfg.xlim = [0.05 1.2];
    cfg.colorbar = 'yes';
    cfg.layout = kit_layout;
    ft_topoplotER(cfg, AVG_TRIALS{tr});

end

Visual stimulus
---------------

Visual stimulus ERP select from the first plot sensors in the Occipital lobe and visualise the averaged ERP from the sensors
- Observe the M100 and M170

![image.png](attachment:51ade515-40aa-4a10-9e7a-60aefd45f228.png)

Select the M100 peak from the GUI then double click the selection to see the sensor topoplot averaged only at the selection peak time.
Observe then that the strongest magnetic field activity is in the Occipital lobe for the M100.

![image2.png](attachment:948bd08f-76d1-4200-bd05-6b74ad68870f.png)

Auditory Stimulus
-----------------

**Right side averaged sensor ERP**

![image5.png](attachment:a9a16a78-2b16-46f9-8c72-2eedeff94545.png)

![image3.png](attachment:ebd37f1e-87f4-49b3-9cd3-d6c1f146aa3e.png)

**Left side averaged sensor ERP**

![image6.png](attachment:19185a92-1a26-4c3f-9f2e-6c8db2bb98c5.png)

![image7.png](attachment:7d2cab22-0ec5-41f5-8592-405d85952db4.png)

Motor Stimulus
--------------

![image9.png](attachment:3c997de9-a14d-4cf2-946e-159cffee000d.png)

Topology at the M100 peak.

![image10.png](attachment:0c911d37-ea21-4c9b-89e4-015fad329be0.png)